In [44]:
import numpy as np
import torch
import torchvision

In [45]:
import torch.nn as nn
import torch.nn.functional as F

# Network Architecture

In [46]:
class synthesisNet(nn.Module):
    
    def __init__(self,pathLength=40):
        super(synthesisNet,self).__init__()
        self.conv1 = nn.Conv2d(1,1,3)
        self.fc1 = nn.Linear((pathLength-2),1)
        
    def forward(self,x):
        x = torch.sigmoid(self.conv1(x))
        x = x.view(-1,self.num_flat_features(x))
        x = torch.sigmoid(self.fc1(x))
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        print(x.size())
        num_features = 1
        for s in size:
            num_features *= s
        print(num_features)
        return num_features

# Data Processing

In [64]:
import pandas as pd
import numpy as np

In [120]:
def getColumnHeader(fileName):
    with open(fileName, 'r') as temp_f:
        col_count = [ len(l.split(",")) for l in temp_f.readlines() ]
    column_names = [i for i in range(0, max(col_count))]
    return column_names

In [135]:
def getRawInputDataDF(fileName):
    columnNames = getColumnHeader(fileName)
    return pd.read_csv(fileName,header=None,names=columnNames)

## Timing 1000ps

In [139]:
core1_prePnR = pd.read_csv("dataCore/core1/prePnR.csv")
core1_postPnR = pd.read_csv("dataCore/core1/postPnR.csv")

In [137]:
core1_inputData = getRawInputDataDF("dataCore/core1/inputFeatureFile.csv")

## Timing 800ps

In [141]:
core2_prePnR = pd.read_csv("dataCore/core2/prePnR.csv")
core2_postPnR = pd.read_csv("dataCore/core2/postPnR.csv")
core2_inputData = getRawInputDataDF("dataCore/core2/inputFeatureFile.csv")

## Timing 600ps

In [142]:
core3_prePnR = pd.read_csv("dataCore/core3/prePnR.csv")
core3_postPnR = pd.read_csv("dataCore/core3/postPnR.csv")
core3_inputData = getRawInputDataDF("dataCore/core3/inputFeatureFile.csv")

## Timing 400ps

In [143]:
core4_prePnR = pd.read_csv("dataCore/core4/prePnR.csv")
core4_postPnR = pd.read_csv("dataCore/core4/postPnR.csv")
core4_inputData = getRawInputDataDF("dataCore/core4/inputFeatureFile.csv")

# Data pre-processing

## Common elements pre and post pnr



In [122]:
import math

In [123]:
def getCommonDF(prePnRDF,postPnRDF):
    prePnRDF = prePnRDF.rename(columns={'pathLength': 'prePnRLen', 'pathIdx': 'pidPrePnR'})
    postPnRDF = postPnRDF.rename(columns={'pathLength': 'postPnRLen', 'pathIdx': 'pidPostPnR'})
    prePnRDF['pkey'] = prePnRDF['startPoint']+","+prePnRDF['endPoint']
    postPnRDF['pkey'] = postPnRDF['startPoint']+","+postPnRDF['endPoint']
    commonKeys = set(prePnRDF['pkey']) & set(postPnRDF['pkey'])
    prePnRDF_common = prePnRDF[prePnRDF['pkey'].isin(commonKeys)]
    postPnRDF_common = postPnRDF[postPnRDF['pkey'].isin(commonKeys)]
    final = prePnRDF_common.merge(postPnRDF_common, on="pkey", how = 'inner')
    final["techmapPD"] = final["techmapPD"]*1000
    return final

In [131]:
def getInputDataAndLabelDF(coreLabelDF,coreInputDF):
    filteredLabelDF = coreLabelDF[['pidPrePnR','techmapPD']]
    filteredInputDF = coreInputDF.rename(columns={0:'pidPrePnR'})
    augmentedFilteredDF = filteredLabelDF.merge(filteredInputDF, on="pidPrePnR", how = 'inner')
    #Labelling top 10% as critical Path
    augmentedFilteredDF.sort_values(by=["techmapPD"],ascending=False)
    augmentedFilteredDF.insert(0,'label',0)
    for i in range(math.floor(0.1*augmentedFilteredDF.shape[0])):
        augmentedFilteredDF.loc[i,'label']= 1
    #Drop unncessary columns
    augmentedFilteredDF = augmentedFilteredDF.drop(columns=['pidPrePnR','techmapPD'])
    return augmentedFilteredDF

## Core 1 Dataset

In [144]:
core1_df = getCommonDF(core1_prePnR,core1_postPnR)
core1_dataset = getInputDataAndLabelDF(core1_df,core1_inputData)

## Core 2 Dataset

In [145]:
core2_df = getCommonDF(core2_prePnR,core2_postPnR)
core2_dataset = getInputDataAndLabelDF(core2_df,core2_inputData)

## Core 3 Dataset

In [147]:
core3_df = getCommonDF(core3_prePnR,core3_postPnR)
core3_dataset = getInputDataAndLabelDF(core3_df,core3_inputData)

## Core 4 Dataset

In [146]:
core4_df = getCommonDF(core4_prePnR,core4_postPnR)
core4_dataset = getInputDataAndLabelDF(core4_df,core4_inputData)

In [170]:
consolidatedData = pd.concat([core1_dataset,core2_dataset,core3_dataset,core4_dataset],axis=0).reset_index(drop=True)

# Dataset pre-processing

In [172]:
# Identify the columns having all the entries as NaN, and drop such columns
count = 0
dropColumnList = []
for col in consolidatedData.columns:
    if(consolidatedData[col].isnull().sum() == len(consolidatedData)):
        count+=1
        dropColumnList.append(col)

In [178]:
consolidatedData = consolidatedData.drop(columns=dropColumnList)

In [179]:
dataLabel = consolidatedData['label']
dataFeatures = consolidatedData.drop(columns=['label'])

In [186]:
numOfSamples = dataLabel.shape[0]
maxLenOfPath = int(len(dataFeatures.columns)/3) # Three feature per gate

In [218]:
inputDataFatures = dataFeatures.to_numpy().reshape(numOfSamples,1,maxLenOfPath,3)

In [219]:
input_ProcessedData=np.nan_to_num(inputDataFatures)

In [211]:
output_labels = dataLabel.to_numpy()

# Dataset normalization

## MinMax scaling

In [220]:
minLoad = np.amin(np.amin(np.array(input_ProcessedData[:,:,:,2])[input_ProcessedData[:,:,:,2] != np.amin(input_ProcessedData[:,:,:,2])]))
maxLoad = np.amax(input_ProcessedData[:,:,:,2])

In [221]:
minTargetCycleTime = np.amin(np.amin(np.array(input_ProcessedData[:,:,:,0])[input_ProcessedData[:,:,:,0] != np.amin(input_ProcessedData[:,:,:,0])]))
maxTargetCycleTime = np.amax(input_ProcessedData[:,:,:,0])

In [225]:
input_ProcessedData[:,:,:,0] = (input_ProcessedData[:,:,:,0] - minTargetCycleTime)/(maxTargetCycleTime-minTargetCycleTime)

In [229]:
input_ProcessedData[:,:,:,2] = (input_ProcessedData[:,:,:,2] - minLoad)/(maxLoad-minLoad)

In [231]:
input_ProcessedData = input_ProcessedData.clip(min=0.0)

## Export data as numpy objects

In [246]:
np.save('inputFeatures.npy',input_ProcessedData)
np.save('outputLabels.npy',output_labels)